In [2]:
import pandas as pd

In [3]:
import pickle

In [4]:
import seaborn as sns
import matplotlib.pyplot as plt

In [5]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge

from sklearn.metrics import root_mean_squared_error

In [6]:
import mlflow


mlflow.set_tracking_uri("sqlite:///mlflow.db")
mlflow.set_experiment("nyc-taxi-experiment")

<Experiment: artifact_location='/workspaces/mlops-zoomcamp/02-experiment-tracking/mlruns/2', creation_time=1747582843607, experiment_id='2', last_update_time=1747582843607, lifecycle_stage='active', name='nyc-taxi-experiment', tags={}>

In [7]:
def read_dataframe(filename):
    df = pd.read_parquet(filename)
    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)
    df = df[(df.duration >=1) & (df.duration <= 60)]
    categorical = ['PULocationID','DOLocationID']
    df[categorical] = df[categorical].astype(str)
    return df

In [8]:
df_train = read_dataframe('./data/green_tripdata_2021-01.parquet')
df_val = read_dataframe('./data/green_tripdata_2021-02.parquet')

In [9]:
df_train['PU_DO'] = df_train['PULocationID'] + df_train['DOLocationID']
df_val['PU_DO'] = df_val['PULocationID'] + df_val['DOLocationID']

In [10]:
categorical = ['PU_DO']#['PULocationID','DOLocationID']
numerical = ['trip_distance']

dv = DictVectorizer()
train_dict = df_train[categorical+numerical].to_dict(orient='records')
X_train = dv.fit_transform(train_dict)

val_dicts = df_val[categorical+numerical].to_dict(orient='records')
X_val = dv.transform(val_dicts)

In [11]:
target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

In [11]:
lr = LinearRegression()
lr.fit(X_train,y_train)
y_pred = lr.predict(X_val)
root_mean_squared_error(y_val, y_pred)

7.787351945269037

In [12]:
lr = Lasso(alpha=0.005)
lr.fit(X_train,y_train)
y_pred = lr.predict(X_val)
root_mean_squared_error(y_val, y_pred)

10.693262833995945

In [ ]:
lr = Ridge(alpha=0.01)
lr.fit(X_train,y_train)
y_pred = lr.predict(X_val)
root_mean_squared_error(y_val, y_pred)

57.00202336636105

In [14]:
with open('./models/lin_reg.bin','wb') as f_out:
    pickle.dump((dv,lr), f_out)

In [12]:
import xgboost as xgb 
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.pyll import scope

In [13]:
train = xgb.DMatrix(X_train, label=y_train)
valid = xgb.DMatrix(X_val, label=y_val)

In [14]:
def objective(params):
    with mlflow.start_run():
        mlflow.set_tag("model", "xgboost")
        mlflow.log_params(params)
        booster = xgb.train(
            params=params,
            dtrain=train,
            num_boost_round=1000,
            evals=[(valid, 'validation')],
            early_stopping_rounds=50
        )
        y_pred = booster.predict(valid)
        rmse = root_mean_squared_error(y_val, y_pred)
        mlflow.log_metric("rmse", rmse)

    return {'loss': rmse, 'status': STATUS_OK}

In [14]:
search_space = {
    'max_depth': scope.int(hp.quniform('max_depth', 4, 100, 1)),
    'learning_rate': hp.loguniform('learning_rate', -3, 0),
    'reg_alpha': hp.loguniform('reg_alpha', -5, -1),
    'reg_lambda': hp.loguniform('reg_lambda', -6, -1),
    'min_child_weight': hp.loguniform('min_child_weight', -1, 3),
    'objective': 'reg:linear',
    'seed': 42
}

best_result = fmin(
    fn=objective,
    space=search_space,
    algo=tpe.suggest,
    max_evals=50,
    trials=Trials()
)

  0%|          | 0/50 [00:00<?, ?trial/s, best loss=?]

/home/codespace/anaconda3/envs/exp-tracking-env/lib/python3.9/site-packages/xgboost/core.py:158: UserWarning: [03:22:02] WARNING: /workspace/src/objective/regression_obj.cu:227: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)



[0]	validation-rmse:9.32546                           
[1]	validation-rmse:7.90350                           
[2]	validation-rmse:7.25066                           
[3]	validation-rmse:6.95333                           
[4]	validation-rmse:6.81010                           
[5]	validation-rmse:6.74277                           
[6]	validation-rmse:6.70146                           
[7]	validation-rmse:6.67829                           
[8]	validation-rmse:6.66470                           
[9]	validation-rmse:6.65269                           
[10]	validation-rmse:6.64904                          
[11]	validation-rmse:6.64157                          
[12]	validation-rmse:6.63831                          
[13]	validation-rmse:6.63494                          
[14]	validation-rmse:6.63210                          
[15]	validation-rmse:6.62955                          
[16]	validation-rmse:6.62617                          
[17]	validation-rmse:6.62414                          
[18]	valid

/home/codespace/anaconda3/envs/exp-tracking-env/lib/python3.9/site-packages/xgboost/core.py:158: UserWarning: [03:23:04] WARNING: /workspace/src/objective/regression_obj.cu:227: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)



: 

In [16]:
import mlflow.xgboost


params = {
    'earning_rate': 0.25354536063020344,
    'max_depth':86 ,
    'min_child_weight': 2.9828515568234,
    'objective': 'reg:linear',
    'reg_alpha': 0.028145387058627437,
    'reg_lambda': 0.30613009300449756,
    'seed': 42,
}

mlflow.xgboost.autolog()
booster = xgb.train(
            params=params,
            dtrain=train,
            num_boost_round=1000,
            evals=[(valid, 'validation')],
            early_stopping_rounds=50
        )

2025/05/19 03:36:44 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID 'd4eaabccc24e4ddeb289e4ce383518bb', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current xgboost workflow


: 